In [ ]:
import itertools
import importlib
from pathlib import Path

import numpy as np

from bokeh.io import output_notebook
output_notebook()

In [ ]:
import dataloader
importlib.reload(dataloader)

minutes_to_bin = 4
data = np.zeros((7, 24 * 60 // minutes_to_bin), dtype=np.float32)
datadir = Path("data")
days = list(sorted(datadir.iterdir()))
for day in days:
    dataloader.rasterize_week(data, str(day))

In [ ]:
max(max(a) for a in data)

In [ ]:
from bokeh.palettes import Dark2_5 as palette
from bokeh.plotting import figure, show, output_file, save

bin_labels = {}
for bin in range((24 * 60) // minutes_to_bin):
    hour = (bin * minutes_to_bin) // 60
    minute = (bin * minutes_to_bin) % 60
    posmod_hour = ((hour + 11) % 12) + 1
    bin_labels[bin] = f"{posmod_hour}:{minute:02} {'am' if hour < 12 else 'pm'}"

p = figure(width=1400, height=800) # , y_axis_type="log")
p.xaxis.major_label_overrides = bin_labels
p.yaxis.visible = False

days = {
    "Monday": "#003c30",
    "Tuesday": "#01665e",
    "Wednesday": "#35978f",
    "Thursday": "#80cdc1",
    "Friday": "#bf812d",
    "Saturday": "#8c510a",
    "Sunday": "#543005"
}

xs = np.arange(0, len(data[0]))
step = max(max(a) for a in data) * 0.3
for weekday, (name, color) in enumerate(days.items()):
    ys = data[weekday] + (6 - weekday) * step
    p.line(x=xs, y=ys, line_width=2, color=color, legend_label=name)
p.legend.location = "top_left"
p.legend.click_policy="hide"
show(p)
if True:
    output_file("traffic.html", title="Home Network Traffic Graph")
    save(p)